In [ ]:
# 0. 필요한 라이브러리 설치 (주석 처리 - 필요 시 주석 해제 후 실행)
# !pip install -q langchain langchain-community langchain-upstage pypdf faiss-cpu python-dotenv

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_upstage import UpstageEmbeddings
# .env 파일에서 환경 변수 로드 (API 키 설정)
# UPSTAGE_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

# 기존: load_dotenv() 후
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY") 
# print(UPSTAGE_API_KEY[30:]) # 로드 확인

# ---
## 0단계: 문서 로드
# ---
# 파일 경로가 프로젝트 루트의 'data' 폴더 안에 있다고 가정합니다.
# 실제 경로에 맞게 '../data/콘텐츠분쟁해결_사례.pdf'를 수정해야 할 수 있습니다.
FILE_PATH = '../data/콘텐츠분쟁해결_사례.pdf'

try:
    # 0단계: 문서 로드
    print("✅ 0단계: 문서 로드 시작...")
    loader = PyPDFLoader(FILE_PATH)
    documents = loader.load()
    print(f"   -> 총 {len(documents)} 페이지의 문서를 로드했습니다.")
except FileNotFoundError:
    print(f"❌ 오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {FILE_PATH}")
    # 파일이 없을 경우 이후 단계를 건너뛰기 위해 종료
    # exit() 
except Exception as e:
    print(f"❌ 문서 로드 중 오류 발생: {e}")
    # exit()

# ---
## 1단계: 문서 분할 설정 및 실행
# ---
print("\n✅ 1단계: 문서 분할(Chunking) 시작...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 법률 사례는 1200-1800자 권장
    chunk_overlap=100,      # 사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",    # 법률 문서 섹션 구분자
        "\n【쟁점사항】",    # 쟁점 부분 구분
        "\n【처리경위】",    # 처리 과정 구분
        "\n【처리결과】",    # 결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ],
    length_function=len
)
split_documents = text_splitter.split_documents(documents)
print(f"   -> 분할된 문서 조각(chunks) 수: {len(split_documents)}개")


# ---
## 2단계: 임베딩 모델 설정 및 벡터스토어 생성
# ---
print("\n✅ 2단계: 임베딩 모델 설정 및 벡터스토어 생성 시작...")
# 2단계: 임베딩 모델 설정
embeddings = UpstageEmbeddings(model="solar-embedding-1-large", upstage_api_key=UPSTAGE_API_KEY)

# FAISS 벡터스토어 생성 및 임베딩
# 이 과정은 시간이 소요될 수 있습니다.
vectorstore = FAISS.from_documents(split_documents, embeddings)
print("   -> FAISS 벡터스토어 생성이 완료되었습니다.")

# ---
## 3단계: 검색기 설정
# ---
print("\n✅ 3단계: 검색기 설정...")
# 3단계: 검색기 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",       # 유사도 검색 사용
    search_kwargs={"k": 3}          # 상위 5개 관련 사례 검색
)
print("   -> 검색기(Retriever)가 설정되었습니다. (검색 방식: similarity, k: 5)")


# ---
## 4단계: LLM 설정
# ---
print("\n✅ 4단계: LLM 설정...")
# 4단계: LLM 설정
llm = ChatUpstage(
        model="solar-pro",
        upstage_api_key=UPSTAGE_API_KEY, # <-- 키 직접 전달
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print("   -> LLM이 설정되었습니다. (모델: solar-pro, 온도: 0.5)")


# ---
## 5단계: 법률 자문 프롬프트 작성
# ---
print("\n✅ 5단계: 법률 자문 프롬프트 작성...")
# 5단계: 법률 자문 프롬프트 작성
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다.
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요.
2. 관련 법령이나 조항이 있다면 명시하세요.
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요.
4. 실무적 해결방안을 단계별로 제시하세요.
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요.

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print("   -> 법률 자문 프롬프트 템플릿 작성이 완료되었습니다.")


# ---
## 6단계: QA 체인 생성
# ---
print("\n✅ 6단계: QA 체인 생성 시작...")
# 6단계: QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 모든 문서를 하나의 프롬프트에 넣어 처리
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}, # 사용자 정의 프롬프트 적용
    return_source_documents=True # 참조 문서도 함께 반환
)
print("   -> RetrievalQA 체인 생성이 완료되었습니다.")


# ---
## 7단계: 테스트 질문 작성 및 실행
# ---
print("\n✅ 7단계: 테스트 질문 작성 및 실행 시작...")
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?"
]

def ask_question_and_print_result(question):
    print(f"\n========================================================")
    print(f"🔍 질문: {question}")
    print(f"========================================================")
    result = None  # result 변수 미리 선언
    try:
        result = qa_chain.invoke({"query": question})  # 또는 {"question": question} 등 프롬프트 input_variables와 일치
        print("✅ [전문 법률 조언]")
        print(result['result'])
    except Exception as e:
        print(f"❌ QA 체인 실행 중 오류 발생: {e}")
    if result is not None:
        print("\n📜 [참조 문서 (Source Documents)]")
        for i, doc in enumerate(result['source_documents']):
            source = doc.metadata.get('source', 'Unknown')
            page = doc.metadata.get('page', 'Unknown')
            print(f"  {i+1}. 파일: {os.path.basename(source)}, 페이지: {page} (점수: {doc.metadata.get('score', 'N/A')})")
    print("--------------------------------------------------------")


    
    # 참조 문서 출력
    print("\n📜 [참조 문서 (Source Documents)]")
    for i, doc in enumerate(result['source_documents']):
        # 메타데이터에서 파일 이름과 페이지 번호 추출 시도
        source = doc.metadata.get('source', 'Unknown')
        page = doc.metadata.get('page', 'Unknown')
        print(f"  {i+1}. 파일: {os.path.basename(source)}, 페이지: {page} (점수: {doc.metadata.get('score', 'N/A')})")
        # 문서 내용 일부를 보여줄 경우 (디버깅 목적)
        # print(f"     내용 요약: {doc.page_content[:150]}...")
    print("--------------------------------------------------------")

# 테스트 질문 실행
for question in test_questions:
    ask_question_and_print_result(question)
    

# ---
## 8단계: 분쟁 유형 분류 함수 (참고용)
# ---
# 이 함수는 체인 실행 전에 질문 유형을 파악하여 프롬프트 또는 검색 전략을 조정하는 데 활용할 수 있습니다.
print("\n✅ 8단계: 분쟁 유형 분류 함수 (참고)")
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

# 분류 함수 테스트
test_query = "온라인 강의 환불 절차를 알려주세요."
print(f"   -> '{test_query}'의 분쟁 유형: {classify_dispute_type(test_query)}")